In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ChatVectorDBChain
from langchain.llms import OpenAI
import os

In [ ]:
class LangChainChat:
    def __init__(self, initial_pdf_path, api_key):
        self.pdf_path = initial_pdf_path
        self.api_key = api_key
        self.pdf_qa = None
        self.pages = None

    def load_pdf(self, pdf_path):
        loader = PyPDFLoader(pdf_path)
        pages = loader.load_and_split()
        return pages

    def process_langchain(self):
        # Create embeddings using OpenAI with the API key
        embeddings = OpenAIEmbeddings(openai_api_key=self.api_key)

        # Create vector store using Chroma
        vectordb = Chroma.from_documents(self.pages, embedding=embeddings, persist_directory=".")

        # Persist vector store
        vectordb.persist()

        # Set the OpenAI API key environment variable
        os.environ["OPENAI_API_KEY"] = self.api_key

        # Create a ChatVectorDBChain from langchain
        self.pdf_qa = ChatVectorDBChain.from_llm(
            OpenAI(temperature=0.9, model_name="gpt-3.5-turbo"),
            vectordb,
            return_source_documents=True
        )

    def start_chat(self):
        while True:
            # Prompt user for a question
            query = input("Ask a question ('exit' to end the chat, 'switch' to change PDF): ")

            # Check if the user wants to exit the chat
            if query.lower() == 'exit':
                break

            # Check if the user wants to switch PDFs
            elif query.lower() == 'switch':
                self.pdf_path = input("Enter the new PDF file path: ")
                try:
                    self.pages = self.load_pdf(self.pdf_path)
                    self.process_langchain()
                    print(f"Switched to {self.pdf_path}")
                except Exception as e:
                    print(f"Error loading PDF: {e}")

            # Query for question answering
            else:
                result = self.pdf_qa({"question": query, "chat_history": ""})
                
                print("Question:")
                print(query)

                # Print the answer
                print("Answer:")
                print(result["answer"],'\n')

In [ ]:
if __name__ == "__main__":
    # Prompt user for the initial PDF file path and API key
    initial_pdf_path = input("Enter the initial PDF file path: ")
    api_key = input("Enter your OpenAI API key: ")

    # Create LangChainChat instance
    chat_instance = LangChainChat(initial_pdf_path, api_key)

    # Load initial PDF and start chat
    try:
        chat_instance.pages = chat_instance.load_pdf(initial_pdf_path)
        chat_instance.process_langchain()
    except Exception as e:
        print(f"Error loading initial PDF: {e}")
        exit()

    # Start the chat loop
    chat_instance.start_chat()